<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-04 10:09:45
-------------------
qualified stocks: 93
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.03 L
Current:  1.31 C
-------------------
Today PnL: -33.20 K (-0.25%)
Current PnL: -16.94 L (-11.8%)
CY Booked + Current PnL: -12.47 L (-8.69%)
-------------------
Total profit:  4.16 L
Total loss:  -21.10 L
-------------------
Total Booked + Current PnL: 14.51 L (12.45%)
Total Booked PnL: 31.45 L (26.99%)
Curr Year Booked PnL: 4.47 L (3.41%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.11 C
Est FTT PnL: 80.04 L (61.02%)
-------------------
Est LTT:  2.73 C
Est LTT PnL: 1.42 C (108.47%)
Deployed:  1.17 C
Current:  1.31 C
CAGR/XIRR %: 8.41%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PIDILITIND,2504.06,3202.00,-16.23,X-LC,43.01,92772.0,17650.0,3284.0,-0.12,23.50,3.54,27.87,9.0,5.37,0.71,16.35,X40,NTT,CHEMICALS
49,LICI,785.05,983.00,-3.65,H-LC,88.17,235848.0,41156.0,7924.0,-0.28,21.14,3.36,25.21,31.0,5.19,1.81,30.42,XY25,NTT,INSURANCE
28,GILLETTE,8109.44,10330.69,12.01,H-SC,92.47,165155.0,27295.0,10471.0,0.70,19.80,6.34,27.39,116.0,2.61,1.27,38.18,X40,ATH,FMCG
4,ADANIPORTS,1103.69,1583.00,1.05,M-LC,60.22,101360.0,22998.0,11038.0,-0.33,29.35,10.89,43.43,70.0,2.08,0.78,35.57,XY24,NTT,MISC
62,RELIANCE,1291.83,1526.00,3.47,X-LC,17.20,153526.0,12717.0,12804.0,0.21,9.03,8.34,18.13,19.0,0.99,1.18,21.22,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,GREENPANEL,375.16,537.00,186.18,M-SC,75.27,127661.0,-55417.0,134402.0,2.13,-30.27,105.28,43.14,191.0,-0.41,0.98,20.08,XY24,NTT,MISC
50,LTIM,5564.16,7197.33,-3.95,X-LC,79.57,185058.0,-15252.0,74042.0,1.57,-7.61,40.01,29.35,3.0,-0.21,1.42,27.85,X200,ATH,IT
55,PFIZER,4245.50,6318.30,14.35,H-SC,98.92,132296.0,34650.0,13031.0,1.03,35.48,9.85,48.82,134.0,2.66,1.02,46.97,X40,ATH,PHARMA
15,BSOFT,628.85,844.81,-25.25,M-SC,34.41,70228.0,-39821.0,77616.0,0.98,-36.18,110.52,34.34,184.0,-0.51,0.54,15.62,XR,ATH,IT
40,INDUSINDBK,1354.50,1800.00,-37.72,H-MC,11.83,92055.0,-62358.0,113145.0,0.83,-40.38,122.91,32.89,106.0,-0.55,0.71,26.76,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SATIN,199.77,274.0,-22.47,H-SC,10.75,152173.0,-43801.0,116625.0,-2.64,-22.35,76.64,37.16,122.0,-0.38,1.17,13.64,XY24,NTT,FINANCE
75,SURYODAY,189.78,240.0,55.70,H-SC,62.37,110774.0,-39152.0,78827.0,-2.32,-26.11,71.16,26.46,129.0,-0.50,0.85,41.91,XR,NTT,BANKS
87,VALIANTORG,512.64,838.0,-375.11,H-SC,100.00,117026.0,-16260.0,100853.0,-2.05,-12.20,86.18,63.47,130.0,-0.16,0.90,96.16,XR,NTT,CHEMICALS
81,TITAGARH,1097.23,1548.0,-12.03,H-SC,91.40,154490.0,-37525.0,116408.0,-1.77,-19.54,75.35,41.08,128.0,-0.32,1.19,30.76,XY24,NTT,ENGINEERING
10,BANDHANBNK,214.25,400.0,-35.06,H-SC,44.09,194474.0,-54056.0,269522.0,-1.39,-21.75,138.59,86.70,115.0,-0.20,1.49,28.65,XY24,NTT,BANKS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1461.46,2275.00,-13.50,X-LC,50.54,277325.0,15724.0,129899.0,0.41,6.01,46.84,55.67,1.0,0.12,2.13,14.02,X40,BTT,IT
80,TCS,3794.03,4998.00,-19.05,X-LC,18.28,263851.0,-32083.0,125989.0,0.22,-10.84,47.75,31.73,2.0,-0.25,2.03,5.61,X40,BTT,IT
50,LTIM,5564.16,7197.33,-3.95,X-LC,79.57,185058.0,-15252.0,74042.0,1.57,-7.61,40.01,29.35,3.0,-0.21,1.42,27.85,X200,ATH,IT
6,ASIANPAINT,2961.56,4250.00,-23.26,X-LC,4.30,152653.0,-48733.0,136350.0,-0.48,-24.20,89.32,43.51,5.0,-0.36,1.17,5.29,X40,BTT,PAINTS
34,HINDUNILVR,2413.81,2723.00,-16.30,X-LC,24.73,262752.0,-7595.0,42224.0,-0.27,-2.81,16.07,12.81,6.0,-0.18,2.02,8.74,XY25,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.3,14.35,H-SC,98.92,132296.0,34650.0,13031.0,1.03,35.48,9.85,48.82,134.0,2.66,1.02,46.97,X40,ATH,PHARMA
85,UNIONBANK,123.87,163.0,-1.84,M-LC,90.32,171244.0,30404.0,14093.0,-0.68,21.59,8.23,31.59,79.0,2.16,1.32,48.75,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,1.05,M-LC,60.22,101360.0,22998.0,11038.0,-0.33,29.35,10.89,43.43,70.0,2.08,0.78,35.57,XY24,NTT,MISC
49,LICI,785.05,983.0,-3.65,H-LC,88.17,235848.0,41156.0,7924.0,-0.28,21.14,3.36,25.21,31.0,5.19,1.81,30.42,XY25,NTT,INSURANCE
56,PIDILITIND,2504.06,3202.0,-16.23,X-LC,43.01,92772.0,17650.0,3284.0,-0.12,23.50,3.54,27.87,9.0,5.37,0.71,16.35,X40,NTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,98.52,M-MC,86.02,225754.0,792.0,166855.0,0.68,0.35,73.91,74.53,170.0,0.00,1.73,31.07,XY24,BTT,STEEL
51,LUXIND,1448.19,2442.00,35.79,M-SC,63.44,141056.0,3478.0,90939.0,-1.18,2.53,64.47,68.62,183.0,0.04,1.08,22.29,X40N,NTT,TEXTILES
55,PFIZER,4245.50,6318.30,14.35,H-SC,98.92,132296.0,34650.0,13031.0,1.03,35.48,9.85,48.82,134.0,2.66,1.02,46.97,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,12.01,H-SC,92.47,165155.0,27295.0,10471.0,0.70,19.80,6.34,27.39,116.0,2.61,1.27,38.18,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,5.05,H-MC,36.56,99790.0,14689.0,28410.0,-0.02,17.26,28.47,50.65,90.0,0.52,0.77,25.44,X40N,ATH,CABLES


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,3.33,H-SC,68.82,116113.0,14109.0,30909.0,-1.13,13.83,26.62,44.14,112.0,0.46,0.89,32.32,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-28.22,M-MC,61.29,140862.0,11710.0,75361.0,-0.65,9.07,53.50,67.42,156.0,0.16,1.08,37.00,XR,NTT,BANKS
38,INDIAMART,2327.09,4911.36,-51.65,H-SC,59.14,125716.0,2380.0,134592.0,0.44,1.93,107.06,111.05,108.0,0.02,0.97,23.18,AR,ATH,MISC
92,WIPRO,243.46,420.00,-10.62,M-LC,37.63,153208.0,2263.0,107184.0,0.36,1.50,69.96,72.51,68.0,0.02,1.18,15.04,XR,NTT,IT
1,AARTIIND,487.04,919.00,36.75,M-SC,51.61,78465.0,-2871.0,75005.0,-0.16,-3.53,95.59,88.69,205.0,-0.04,0.60,32.31,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MASFIN,326.60,399.5,-24.12,H-SC,64.52,86400.0,-11580.0,33454.0,-1.35,-11.82,38.72,22.32,127.0,-0.35,0.66,25.31,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,1.05,M-LC,60.22,101360.0,22998.0,11038.0,-0.33,29.35,10.89,43.43,70.0,2.08,0.78,35.57,XY24,NTT,MISC
84,UJJIVANSFB,52.77,60.0,35.36,M-SC,53.76,117423.0,-25056.0,44574.0,-0.05,-17.59,37.96,13.70,221.0,-0.56,0.90,38.59,X40N,NTT,BANKS
87,VALIANTORG,512.64,838.0,-375.11,H-SC,100.00,117026.0,-16260.0,100853.0,-2.05,-12.20,86.18,63.47,130.0,-0.16,0.90,96.16,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.3,14.35,H-SC,98.92,132296.0,34650.0,13031.0,1.03,35.48,9.85,48.82,134.0,2.66,1.02,46.97,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-375.11,H-SC,100.00,117026.0,-16260.0,100853.0,-2.05,-12.20,86.18,63.47,130.0,-0.16,0.90,96.16,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.30,14.35,H-SC,98.92,132296.0,34650.0,13031.0,1.03,35.48,9.85,48.82,134.0,2.66,1.02,46.97,X40,ATH,PHARMA
7,ASIANTILES,75.41,137.00,6450.00,M-SC,97.85,71891.0,-21919.0,98541.0,-0.31,-23.37,137.07,81.67,214.0,-0.22,0.55,39.22,XR,NTT,CERAMICS
74,STARHEALTH,551.56,761.00,5.39,M-SC,96.77,172389.0,-27827.0,103847.0,0.50,-13.90,60.24,37.97,195.0,-0.27,1.32,38.41,XY24,NTT,INSURANCE
77,TANLA,917.30,1963.11,-34.59,H-SC,95.70,160632.0,-78783.0,351736.0,0.11,-32.91,218.97,114.01,113.0,-0.22,1.23,47.32,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.69
1,25,42.02
2,50,70.54


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    31.48
XY25    14.58
X40     14.47
AR      13.43
X40N    13.20
XR      10.29
X200     1.42
SR       1.12
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.14
X-LC    18.72
M-SC    17.39
H-MC     8.49
M-LC     7.67
H-LC     7.60
X-MC     5.56
M-MC     5.50
X-SC     2.73
L-SC     2.25
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.73,-0.82,32.11
IT,10.18,-17.74,81.69
BANKS,9.65,-11.61,62.93
MISC,6.41,-29.71,87.08
FINANCE,5.80,-26.51,86.59
INSURANCE,5.30,3.45,25.48
PAINTS,5.25,-20.34,55.95
ELECTRICAL,5.11,-5.71,51.57
AUTO,4.30,-11.84,60.70


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2878934.0,26
AR,1235650.0,12
X40N,1226761.0,17
XR,1111245.0,14
X40,797612.0,11
XY25,481424.0,10
SR,198561.0,2
X200,74042.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2265090.0,21
M-SC,2049673.0,21
X-LC,873139.0,12
M-MC,550697.0,5
H-MC,523234.0,8
X-SC,352455.0,4
X-MC,310416.0,5
M-LC,309313.0,6
L-SC,304069.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      947078.0      6
M-SC       XY24      752558.0      6
H-SC       AR        546550.0      3
           X40N      503917.0      6
X-LC       X40       484592.0      5
M-SC       X40N      476143.0      6
M-MC       XY24      393118.0      3
M-SC       XR        293892.0      4
H-SC       XR        244043.0      4
M-SC       X40       225277.0      2
H-LC       AR        212372.0      3
M-LC       XY24      187729.0      4
H-MC       XY24      177416.0      2
X-LC       AR        174593.0      2
L-SC       XR        159178.0      2
X-LC       XY25      139912.0      4
X-MC       XY24      137529.0      2
M-SC       XY25      122170.0      1
X-SC       X40N      120893.0      2
           XR        118442.0      1
H-MC       XR        113145.0      1
X-SC       SR        113120.0      1
M-LC       XR        107184.0      1
X-MC       X40N       97398.0      2
M-SC       AR         94192.0      1
           SR         85441.0      1
L-SC       XY24       82248.0      1
M-MC       AR         82218.0      1
H-MC       XY25       76940.0      1
X-MC       XY25       75489.0      1
M-MC       XR         75361.0      1
X-LC       X200       74042.0      1
H-MC       X40        64241.0      2
           AR         63082.0      1
L-SC       AR         62643.0      1
L-LC       XY25       44589.0      1
H-MC       X40N       28410.0      1
H-SC       X40        23502.0      2
M-LC       XY25       14400.0      1
H-LC       XY25        7924.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
